In [ ]:
import gradio as gr
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load preprocessors and model
with open('random_forest.pkl', 'rb') as f:
    model = pickle.load(f)

# Initialize preprocessors
le_gender = LabelEncoder().fit(['Male', 'Female'])
le_married = LabelEncoder().fit(['Yes', 'No'])
le_dependents = LabelEncoder().fit(['0', '1', '2', '3+'])
le_education = LabelEncoder().fit(['Graduate', 'Not Graduate'])
le_self_employed = LabelEncoder().fit(['Yes', 'No'])
le_property_area = LabelEncoder().fit(['Urban', 'Rural', 'Semiurban'])
scaler = StandardScaler()

# Load preprocessed data for scaler
df = pd.read_csv('preprocessed_loan_data.csv')
X = df.drop('Loan_Status', axis=1)
scaler.fit(X)

def predict_loan(gender, married, dependents, education, self_employed, applicant_income, coapplicant_income, loan_amount, loan_amount_term, credit_history, property_area):
    # Prepare input data
    input_data = pd.DataFrame({
        'Gender': [le_gender.transform([gender])[0]],
        'Married': [le_married.transform([married])[0]],
        'Dependents': [le_dependents.transform([dependents])[0]],
        'Education': [le_education.transform([education])[0]],
        'Self_Employed': [le_self_employed.transform([self_employed])[0]],
        'ApplicantIncome': [applicant_income],
        'CoapplicantIncome': [coapplicant_income],
        'LoanAmount': [loan_amount],
        'Loan_Amount_Term': [loan_amount_term],
        'Credit_History': [credit_history],
        'Property_Area': [le_property_area.transform([property_area])[0]]
    })
    
    # Scale numerical features
    input_data_scaled = scaler.transform(input_data)
    
    # Predict
    prediction = model.predict(input_data_scaled)[0]
    
    # Output result
    if prediction == 1:
        return gr.HTML("<h2 style='color: green; text-align: center;'>APPROVED! 🎉 Your loan is good to go!</h2>")
    else:
        return gr.HTML("<h2 style='color: red; text-align: center;'>NOT APPROVED! 😡 *Italian chef snaps spaghetti* No loan for you!</h2>")

# Create Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Loan Approval Prediction 🏦")
    gr.Markdown("Fill in the details below to check if your loan will be approved!")
    
    with gr.Row():
        gender = gr.Dropdown(choices=['Male', 'Female'], label="Gender")
        married = gr.Dropdown(choices=['Yes', 'No'], label="Married")
        dependents = gr.Dropdown(choices=['0', '1', '2', '3+'], label="Dependents")
    with gr.Row():
        education = gr.Dropdown(choices=['Graduate', 'Not Graduate'], label="Education")
        self_employed = gr.Dropdown(choices=['Yes', 'No'], label="Self Employed")
        property_area = gr.Dropdown(choices=['Urban', 'Rural', 'Semiurban'], label="Property Area")
    with gr.Row():
        applicant_income = gr.Number(label="Applicant Income")
        coapplicant_income = gr.Number(label="Co-applicant Income")
        loan_amount = gr.Number(label="Loan Amount")
    with gr.Row():
        loan_amount_term = gr.Number(label="Loan Amount Term (months)")
        credit_history = gr.Dropdown(choices=[0, 1], label="Credit History (0 = Bad, 1 = Good)")
    
    submit_button = gr.Button("Check Loan Status")
    output = gr.HTML()
    
    submit_button.click(
        fn=predict_loan,
        inputs=[gender, married, dependents, education, self_employed, applicant_income, coapplicant_income, loan_amount, loan_amount_term, credit_history, property_area],
        outputs=output
    )

demo.launch()

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


c:\Python311\Lib\site-packages\gradio\analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
